In [9]:

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import pandas as pd
import random

In [16]:
# Function to get error code based on the error message
def get_error_code(error_message):
    if 'billed customer' in error_message:
        return 1
    elif 'Phone number not valid' in error_message:
        return 3
    elif 'no longer in service' in error_message:
        return 2
    elif 'No errors, it is a prepaid number' in error_message:
        return 0
    else:
        return -1  # Unknown error message

In [28]:
# Read phone numbers from file and remove country code
with open('USCCphonenumber.txt', 'r') as file:
    phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][1:100]
phone_numbers

# Initialize WebDriver
options = webdriver.ChromeOptions()
options.headless = True  
driver = webdriver.Chrome(options=options)
driver.get('https://prepaid.uscellular.com/refill/dashboard')
consecutive_error_count = 0


results = []

try:
    # Reject cookies
    WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
    ).click()
    i=1
    # for index, phone_number in phone_numbers:
    for phone_number in phone_numbers:
        print(i)
        i+=1
        # phone number input field
        phone_input_field = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, "phone"))
        )
        phone_input_field.click()  # Focus on the input field

        # Clear the input field 
        phone_input_field.send_keys(Keys.CONTROL + "a")
        phone_input_field.send_keys(Keys.DELETE)

        # Input phone number 
        for char in phone_number:
            phone_input_field.send_keys(char)
            time.sleep(random.uniform(0.3, 1))  # Small delay between each character

        # default error message
        error_message = "No errors, it is a prepaid number"
        error_code = get_error_code(error_message)  # Get error code

        # Check for an error message, wait up to 5 seconds
        try:
            error_message_xpath = "/html/body/div[1]/main/div[1]/div/div[2]/div/div[2]/div[1]/div/div[2]/div/div/div/div/form/div[1]/div[1]/div[2]"
            error_message_element = WebDriverWait(driver, 60).until(
                EC.visibility_of_element_located((By.XPATH, error_message_xpath))
            )
            if error_message_element.text.strip():  # If there's text in the error element
                error_message = error_message_element.text
                error_code = get_error_code(error_message)  # Update error code
        except TimeoutException:
            pass

        if error_code != 0:
            consecutive_error_count += 1
        else:
            consecutive_error_count = 0  # Reset count if error_code is 3

        # Check if the last three error codes were not equal to 3
        if consecutive_error_count == 2:
            driver.quit()  # Close the browser
            time.sleep(random.uniform(30, 60)) # Wait for 20 seconds
            
            # Reset the counter
            consecutive_error_count = 0
            
            # Re-initialize the WebDriver to continue
            driver = webdriver.Chrome(options=options)
            driver.get('https://prepaid.uscellular.com/refill/dashboard')
            WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.ID, "onetrust-reject-all-handler"))
            ).click()


        results.append((phone_number, error_code))
        # driver.refresh()
        # next number
        # phone_input_field.send_keys(Keys.CONTROL + "a")
        # phone_input_field.send_keys(Keys.DELETE)

        # if (index + 1) % 5 == 0:
        #     time.sleep(20)        

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # close the browser.
    driver.quit()


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


KeyboardInterrupt: 

In [29]:
# Convert the results to a DataFrame 
df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
print(df_results)

   Phone Number  Error Code
0    9205407874           2
1    5158021011           1
2    8653633901           1
3    4349600102           1
4    4143137825           1
5    2523124309           1
6    2767243709           2
7    4029427871           3
8    3046980011           1
9    2623252695           1
10   3196662892           3
11   6033814025           2
12   5158224253           1
13   3192155756           2
14   7156794671           1
15   7077271466           3
16   5312803811           3
17   2077624461           2
18   2074969624           1
19   2079244934           0
20   4346608881           1
21   3046575912           1
22   4022155545           2
23   3193613565           1
24   4173173108           2


In [11]:
# df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
# df_results.to_csv('query_result.csv', index=True)  
# print("Results have been saved to query_result.csv")


Results have been saved to query_result.csv


In [37]:
# with open('USCCphonenumber.txt', 'r') as file:
#     phone_numbers = [line.strip()[1:] if line.strip().startswith('1') else line.strip() for line in file][10:15]
# phone_numbers

['2623252695', '3196662892', '6033814025', '5158224253', '3192155756']

In [30]:
import os

csv_file_path = 'uscs_query_result.csv'

df_results = pd.DataFrame(results, columns=['Phone Number', 'Error Code'])
if os.path.exists(csv_file_path):
    df_existing = pd.read_csv(csv_file_path)
    
    last_index = df_existing.index[-1] if not df_existing.empty else -1

    df_results.index = range(last_index + 1, last_index + 1 + len(df_results))
    
    # Append the new results to the existing DataFrame
    df_final = df_existing.append(df_results)
    df_final = df_final.iloc[last_index + 1:,1:]
else:
    df_final = df_results
    df_final.index = range(len(df_final))

# Output to CSV, without overwriting existing data and with continuous indexing
df_final.to_csv(csv_file_path, index_label='Index', mode='w' if not os.path.exists(csv_file_path) else 'a', header=not os.path.exists(csv_file_path))

print(f"Results have been appended to {csv_file_path}")


Results have been appended to uscs_query_result.csv
